# Загрузка модели и подготовка сабмита

In [8]:
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn.functional as F
import torch.optim as optim


from datetime import datetime
from torch import nn
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import MessagePassing, DataParallel
from torch_scatter import scatter
from torch.utils.tensorboard import SummaryWriter

from DataClasses import lmdb_dataset, Dataset, DataListLoader

In [9]:
import sys
sys.path.append(os.path.expanduser('/home/jovyan/ocp-airi/ocpmodels/models'))
sys.path.append(os.path.expanduser('/home/jovyan/ocp-airi/'))

from spinconv import spinconv

Для каждой модели надо написать свою фукнцию preproceccing(), но в обязательно, чтобы в массиве keys был элемент 'sid'

In [10]:
def preprocessing(system):
    keys = ['sid','pos', 'atomic_numbers', 'cell', 'natoms']
    features_dict = {}
    for key in keys:
        features_dict[key] = system[key]
    return Data(**features_dict)

In [11]:
def inference(model, iterator):
    y = torch.tensor([])
    model.eval()

    with torch.no_grad():
        for index, data_list in enumerate(tqdm(iterator)):
            sids = torch.tensor([[data.sid] for data in data_list])
            predictions = model(data_list).squeeze()
            predictions = predictions.reshape((predictions.shape[0], 1)) 

            mini_submit = torch.cat((sids, predictions.to('cpu')), dim=1)
            y = torch.cat((y, mini_submit))

    return y

In [12]:
batch_size = 2
num_workers = -1

target_col = 'y_relaxed'
features_cols = ['feature_1']

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
print(device)

cuda


Здесь мы задаем адрес чекпоинта модели и адрес датасета. Надо вставить их адреса

In [16]:
test_dataset_link = "dataset_adress"
model_adress = "your_model_adress"

В этой клетке надо раскомментить первую строчку и закомментить вторую

In [17]:
# train_dataset_file_path= os.path.expanduser(test_dataset_link)
train_dataset_file_path= os.path.expanduser("../../ocp_datasets/data/is2re/10k/train/data_mod2.lmdbz")

training_set = Dataset(train_dataset_file_path, features_cols, target_col, preprocessing=preprocessing)
training_generator = DataListLoader(training_set, batch_size=batch_size)

В этой клетке надо раскомментить первую строчку и закомментить вторую

In [18]:
# model = torch.load(model_adress)
model = spinconv(None, None, 1, otf_graph=True, regress_forces=False)
model = DataParallel(model)

#переносим на куду если она есть
model = model.to(device)

In [19]:
%%time

# print(f'Start training model {str(model)}')
submission = inference(model, training_generator)

  0%|          | 0/5000 [00:00<?, ?it/s]

ml-test-server-0:78871:78871 [0] NCCL INFO Bootstrap : Using [0]eth0:10.233.122.86<0>
ml-test-server-0:78871:78871 [0] NCCL INFO NET/Plugin : No plugin found (libnccl-net.so), using internal implementation
ml-test-server-0:78871:78871 [0] NCCL INFO NET/IB : No device found.
ml-test-server-0:78871:78871 [0] NCCL INFO NET/Socket : Using [0]eth0:10.233.122.86<0>
ml-test-server-0:78871:78871 [0] NCCL INFO Using network Socket
NCCL version 2.7.8+cuda10.2
ml-test-server-0:78871:78947 [1] NCCL INFO threadThresholds 8/8/64 | 16/8/64 | 8/8/64
ml-test-server-0:78871:78947 [1] NCCL INFO Trees [0] -1/-1/-1->1->0|0->1->-1/-1/-1 [1] -1/-1/-1->1->0|0->1->-1/-1/-1 [2] -1/-1/-1->1->0|0->1->-1/-1/-1 [3] 0/-1/-1->1->-1|-1->1->0/-1/-1 [4] 0/-1/-1->1->-1|-1->1->0/-1/-1 [5] 0/-1/-1->1->-1|-1->1->0/-1/-1 [6] -1/-1/-1->1->0|0->1->-1/-1/-1 [7] -1/-1/-1->1->0|0->1->-1/-1/-1 [8] -1/-1/-1->1->0|0->1->-1/-1/-1 [9] 0/-1/-1->1->-1|-1->1->0/-1/-1 [10] 0/-1/-1->1->-1|-1->1->0/-1/-1 [11] 0/-1/-1->1->-1|-1->1->0/-1/-1
m

100%|██████████| 5000/5000 [13:07<00:00,  6.35it/s]

CPU times: user 15min 31s, sys: 4min, total: 19min 32s
Wall time: 13min 7s


In [20]:
# submission = np.random.randn(2, 5)
submission

tensor([[2472718.0000,     3.3918],
        [1747243.0000,     3.3450],
        [1824821.0000,     2.6289],
        ...,
        [1314291.0000,     3.1283],
        [1972741.0000,     3.8214],
        [1189586.0000,     4.4791]])

In [29]:
np.savez("submission.npz", challenge_ids=submission[:, 0].int(), challenge_energy=submission[:, 1])

In [30]:
sub = np.load("submission.npz")
sub['challenge_ids']

array([2472718, 1747243, 1824821, ..., 1314291, 1972741, 1189586],
      dtype=int32)

In [31]:
sub['challenge_energy']

array([3.3918228, 3.3449738, 2.6289148, ..., 3.128327 , 3.8213568,
       4.479097 ], dtype=float32)